In [ ]:
import pandas as pd
import numpy as np

In [2]:
pheno = pd.read_csv('/data5/16p12_RNA/scripts/16p12.2_rnaseq_analysis/data/pheno_final.tsv', sep='\t')

samples = list(pheno.subject.unique())

In [3]:
filename = '/data5/16p12_RNA/scripts/cnv_parse/output/merged/allsamples.annotated.tsv'

df_small = pd.read_csv(filename, sep='\t')
df_small = df_small[df_small.intra_cohort_count < 10]

In [4]:
filename = '/data5/16p12_RNA/scripts/cnv_parse/output/merged_greater_50/merged.tsv'

df_large = pd.read_csv(filename, sep='\t')
df_large['intra_cohort_count'] = '.'
df_large['intra_cohort_freq'] = '.'

In [5]:
cnvs = df_small.append(df_large)
cnvs = cnvs[cnvs.subject.isin(pheno.subject)].copy()

cnvs = cnvs.drop_duplicates()

cnvs = cnvs.reset_index(drop=True)



In [6]:
# annotate ensembl

keep = pd.read_csv('/data5/16p12_RNA/scripts/16p12.2_rnaseq_analysis/outlier_expression_analysis/keep.tsv',
                   sep='\t')

mapp = pd.read_csv('/data5/16p12_RNA/scripts/16p12.2_rnaseq_analysis/data/gene_names_mapping_new.tsv', sep='\t')
mapp = mapp.set_index('ensembl', drop=False)
mapp = mapp.loc[keep.ensembl]
mapp['chrom'] = mapp.chromosome.apply(lambda s: 'chr{}'.format(s) if s != 'MT' else 'chrM')
mapp.start = mapp.start.astype(int)
mapp.end = mapp.end.astype(int)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
cnvs['ensembl'] = ''
for i in cnvs.index[:]:
    chrom = cnvs.at[i, 'chrom']
    start = cnvs.at[i, 'start']
    end = cnvs.at[i, 'end']

    mappc = mapp[mapp.chrom == chrom]

    min_end = mappc.end.apply(lambda x: min(x, end))
    max_start = mappc.start.apply(lambda x: max(x, start))

    overlap = (min_end - max_start) > 0

    if overlap.sum() > 0:
        mappo = mappc[overlap]  
#         print(i, chrom, start, end, mappo.shape)

        for j in mappo.index:
            if cnvs.at[i, 'ensembl'] == '':
                cnvs.at[i, 'ensembl'] =  mappo.at[j, 'ensembl'] 
            else:
                cnvs.at[i, 'ensembl'] = cnvs.at[i, 'ensembl'] + ';'+ mappo.at[j, 'ensembl'] 

In [8]:


cnvs = cnvs[cnvs.ensembl != '']

cnvs.shape

(926, 12)

In [9]:
cnvs.to_csv('output/merged/allsamples.lcl_expressed_structural_variants.tsv', sep='\t')